### **Import Library**

Sebelum membuat Transformer from scratch, import library terlebih dahulu

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

/home/soic/labby/modul-kecerdasan-buatan/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


### **Positional Encoding**

Karena Transformer tidak mengandalkan urutan alami dari data (seperti urutan waktu pada RNN), informasi posisi token ditambahkan secara eksplisit melalui positional encoding. Vektor ini menggunakan fungsi sinus dan cosinus untuk memberikan “koordinat” yang unik pada tiap token sehingga model dapat memahami urutan kata dalam kalimat. 

Setiap token dalam input ditambahkan dengan vektor posisi yang dihitung menggunakan fungsi sinus dan cosinus. Vektor ini memiliki dimensi yang sama dengan embedding sehingga penjumlahan antara embedding dan positional encoding dapat dilakukan secara langsung.

Positional encoding memberikan representasi unik berbasis pola periodik, mirip dengan bagaimana gelombang sinusoidal dapat merepresentasikan informasi dalam sinyal, sehinnga bisa dianggap seperti memberikan `koordinat` untuk setiap token. Kenapa fungsi sinus dan cosinus? Karena Penggunaan fungsi sinus dan cosinus memastikan bahwa posisi relatif antara kata-kata tetap terjaga, bahkan ketika panjang urutan bervariasi. Hal ini penting karena Transformer tidak memiliki mekanisme memori seperti RNN.

Rumus dari Postional Encoding adalah sebagai berikut:
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

- `d` adalah dimensi embedding
- `pos` adalah index dari posisi
- `i` adalah index dari dimensi

In [11]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        # Buat tensor posisi dengan ukuran (max_len, d_model)
        pe = torch.zeros(max_len, d_model)
        print(pe.shape)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        print(position.shape)
        # Hitung pembagi frekuensi
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Aplikasi rumus sinus dan cosinus
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # Tambahkan dimensi batch agar dapat langsung ditambahkan ke embedding
        pe = pe.unsqueeze(0)  # Shape: [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return x

In [12]:
# Contoh penggunaan:
d_model = 512
pe_layer = PositionalEncoding(d_model)
x = torch.zeros(1, 10, d_model)  # Misal sequence length 10
x_pos = pe_layer(x)
print("Shape input setelah positional encoding:", x_pos.shape)

torch.Size([5000, 512])
torch.Size([5000, 1])
Shape input setelah positional encoding: torch.Size([1, 10, 512])
